In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import urllib
import os.path as op
from pathlib import Path
import re
import unicodedata

### Create ID

In [2]:
new_order = ['id', 'category', 'url', 'title', 'image', 'ingredients', 'steps']

def create_id(data):
    temp = data['url'].split('/')
    return temp[-2] + '-' + temp[-1] 

df_ayam = pd.read_json("raw/cookpad_recipe_ayam.json")
df_ayam['id'] = df_ayam.apply(create_id, axis=1)
df_ayam = df_ayam[new_order]
df_ayam.to_json('clean/cookpad_recipe_ayam.json', orient='records', indent=2)
df_ayam

df_other = pd.read_json("raw/cookpad_recipe_other.json")
df_other['id'] = df_other.apply(create_id, axis=1)
df_other = df_other[new_order]
df_other.to_json('clean/cookpad_recipe_other.json', orient='records', indent=2)
df_other

,id,category,url,title,image,ingredients,steps
0,resep-24976347,daging%20sapi,https://cookpad.com/id/resep/24976347,Lapis Daging Surabaya Empuk dan Gurih,https://img-global.cpcdn.com/recipes/95303d4a3...,"[1/2 kg Daging Sapi Has Dalam, 4 sdm Minyak, 5...","[{'text': 'Bismillah,,, siapkan seluruh bahan ..."
1,resep-24983951,daging%20sapi,https://cookpad.com/id/resep/24983951,Kalio Daging khas Minang,https://img-global.cpcdn.com/recipes/1008c1af5...,"[1 kg daging sapi, 100 gr cabe giling, 1500 ml...",[{'text': 'Haluskan bumbu halus. Campur semua ...
2,resep-24973685,daging%20sapi,https://cookpad.com/id/resep/24973685,1150. Soto Daging 53010,https://img-global.cpcdn.com/recipes/5db4000a8...,"[250 gr Daging Sapi, 800 ml Air, 4 siung Bawan...","[{'text': 'Cuci bersih daging, potong 2-3 bag'..."
3,resep-24983605,daging%20sapi,https://cookpad.com/id/resep/24983605,Gepuk Daging,https://img-global.cpcdn.com/recipes/f6285e575...,[Secukupnya daging sapi (yang sudah direbus/di...,[{'text': 'Potong daging yang sudah direbus se...
4,resep-24816201,daging%20sapi,https://cookpad.com/id/resep/24816201,Kalio daging khas Minangkabau,https://img-global.cpcdn.com/recipes/5cfea2161...,"[2 kg daging sapi tanpa lemak, 2 bungkus bumbu...","[{'text': 'Tumis bawang dalam sedikit minyak, ..."
...,...,...,...,...,...,...,...
4391,resep-17283878,sagu%20keju,https://cookpad.com/id/resep/17283878,Sagu keju,https://img-global.cpcdn.com/recipes/b7bb188fd...,"[140 gr gula pasir, 150 gr butter margarin, 1 ...",[{'text': 'Gula dan butter margarin di mix sam...
4392,resep-17282946,sagu%20keju,https://cookpad.com/id/resep/17282946,Sagu Keju,https://img-global.cpcdn.com/recipes/462081d0a...,"[60 gr Gula pasir (Sy gula halus), 50 gr Buter...",[{'text': 'Siapkan bahan bahan terlebih dahulu...
4393,resep-17280490,sagu%20keju,https://cookpad.com/id/resep/17280490,Sagu keju,https://img-global.cpcdn.com/recipes/06d1f64df...,"[60 gr gula pasir, 50 gr butter, 75 gr margati...",[{'text': 'Siapkan bahan semua nya sangray sag...
4394,resep-17277173,sagu%20keju,https://cookpad.com/id/resep/17277173,Sagu keju,https://img-global.cpcdn.com/recipes/77332add1...,"[450 gr sagu tani, 2 butir kuning telur, 250 g...",[{'text': 'Mixer butter dan gula halus sampai ...


### Download image from url

In [3]:
# df_ayam = pd.read_json("clean/cookpad_recipe_ayam.json")
# df_other = pd.read_json("clean/cookpad_recipe_other.json")

# # get url
# image_ayam = dict(zip(df_ayam['id'], df_ayam['image']))
# image_other = dict(zip(df_other['id'], df_other['image']))
# image_urls = {**image_ayam, **image_other}

# # download image from url
# def download_image_from_url(image_urls: dict, output_dir="../assets/recipe_images", timeout=20):
#     os.makedirs(output_dir, exist_ok=True)

#     with requests.Session() as sess:
#         sess.headers.update({"User-Agent": "Mozilla/5.0 (image-downloader)"})

#         for rid, url in image_urls.items():
#             try:
#                 # get ext from url
#                 path = urllib.parse.urlparse(url).path
#                 ext = op.splitext(path)[1]
#                 if not ext:  # fallback
#                     ext = ".jpg"

#                 # filename
#                 fname = f"{rid}{ext}"
#                 fpath = op.join(output_dir, fname)

#                 # skip image if already downloaded
#                 if os.path.exists(fpath):
#                     print(f"Skipped (already exists): {Path(fpath).as_posix()}")
#                     continue

#                 resp = sess.get(url, stream=True, timeout=timeout)
#                 resp.raise_for_status()

#                 with open(fpath, "wb") as f:
#                     for chunk in resp.iter_content(chunk_size=8192):
#                         if chunk:
#                             f.write(chunk)

#                 print(f"Downloaded: {fpath}")

#             except requests.exceptions.RequestException as e:
#                 print(f"Error downloading {url} (id={rid}): {e}")
#             except Exception as e:
#                 print(f"Unexpected error for id={rid}, url={url}: {e}")

# download_image_from_url(image_urls)

### Cleaning Emoji

In [4]:
df_ayam = pd.read_json("clean/cookpad_recipe_ayam.json")
df_other = pd.read_json("clean/cookpad_recipe_other.json")
df_ayam.head()

,id,category,url,title,image,ingredients,steps
0,resep-24955519,ayam,https://cookpad.com/id/resep/24955519,74. Egg Chicken Roll,https://img-global.cpcdn.com/recipes/f8ec21222...,"[1 butir telur, 1 butir kuning telur, 60 gr te...","[{'text': 'Siapkan semua bahan. Bahan kulit, b..."
1,resep-24983411,ayam,https://cookpad.com/id/resep/24983411,Ayam Panggang Madu Ketumbar,https://img-global.cpcdn.com/recipes/87e2dd042...,"[500-600 gr Ayam, potong², 50 gr minyak untuk ...","[{'text': 'Panaskan minyak, masukkan bawang me..."
2,resep-24975670,ayam,https://cookpad.com/id/resep/24975670,Ayam Asam Manis Pedas,https://img-global.cpcdn.com/recipes/07575109e...,[secukupnya Paha ayam dan 2 potong daging ayam...,"[{'text': 'Siapkan daging ayam. Pesiang, cuci ..."
3,resep-24983731,ayam,https://cookpad.com/id/resep/24983731,Ayam panggang teflon,https://img-global.cpcdn.com/recipes/ab8f2174a...,"[1 kg ayam negri (dipotong 10), 1 buah jeruk n...","[{'text': 'Cuci bersih ayam pada air mengalir,..."
4,resep-24975281,ayam,https://cookpad.com/id/resep/24975281,Chicken Spicy Wings,https://img-global.cpcdn.com/recipes/e9ff08c91...,"[2 kg sayap ayam, Secukupnya minyak untuk meng...","[{'text': 'Cuci bersih sayap ayam, potong jadi..."


In [5]:
# keep letter only in title
def clean_title(title_data):
    text = re.sub(r'[^A-Za-z]+', ' ', title_data)
    return re.sub(r'\s+', ' ', text).strip()

df_ayam['title'] = df_ayam['title'].apply(clean_title)
df_other['title'] = df_other['title'].apply(clean_title)

In [6]:
# delete emoji string
def remove_escaped_unicode(text):
    # remove emoji
    s = re.sub(r'\\U([0-9A-Fa-f]{8})', lambda m: chr(int(m.group(1), 16)), str(text))
    s = re.sub(r'\\u([0-9A-Fa-f]{4})',  lambda m: chr(int(m.group(1), 16)), s)
    s = re.sub(r'[\U00010000-\U0010FFFF]', ' ', s)
    # remove non ascii data
    s = unicodedata.normalize('NFKD', str(s))
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')
    return ' '.join(s.split())

# clean every text in ingredients
def clean_ingredients(data):
    if len(data) == 0:
        return data
    
    if isinstance(data, list):
        return [remove_escaped_unicode(x) for x in data]
    
    return remove_escaped_unicode(data)

# clean every text in steps
def clean_steps(data):
    if len(data) == 0:
        return data
    
    if isinstance(data, list):
        out = []
        for item in data:
            if isinstance(item, dict):
                item = dict(item)
                if 'text' in item:
                    item['text'] = remove_escaped_unicode(item['text'])
                out.append(item)
            else:
                out.append(item)
        return out
    
    return data

# implement
df_ayam['ingredients'] = df_ayam['ingredients'].map(clean_ingredients)
df_ayam['steps'] = df_ayam['steps'].map(clean_steps)
df_other['ingredients'] = df_other['ingredients'].map(clean_ingredients)
df_other['steps'] = df_other['steps'].map(clean_steps)

In [7]:
df_ayam.to_json('clean/cookpad_recipe_ayam.json', orient='records', indent=2)
df_other.to_json('clean/cookpad_recipe_other.json', orient='records', indent=2)

### Cleaning words

In [21]:
df_ayam = pd.read_json("clean/cookpad_recipe_ayam.json")
df_other = pd.read_json("clean/cookpad_recipe_other.json")
df_ayam.head()

,id,category,url,title,image,ingredients,steps
0,resep-24955519,ayam,https://cookpad.com/id/resep/24955519,Egg Chicken Roll,https://img-global.cpcdn.com/recipes/f8ec21222...,"[1 butir telur, 1 butir kuning telur, 60 gr te...","[{'text': 'Siapkan semua bahan. Bahan kulit, b..."
1,resep-24983411,ayam,https://cookpad.com/id/resep/24983411,Ayam Panggang Madu Ketumbar,https://img-global.cpcdn.com/recipes/87e2dd042...,"[500-600 gr Ayam, potong2, 50 gr minyak untuk ...","[{'text': 'Panaskan minyak, masukkan bawang me..."
2,resep-24975670,ayam,https://cookpad.com/id/resep/24975670,Ayam Asam Manis Pedas,https://img-global.cpcdn.com/recipes/07575109e...,[secukupnya Paha ayam dan 2 potong daging ayam...,"[{'text': 'Siapkan daging ayam. Pesiang, cuci ..."
3,resep-24983731,ayam,https://cookpad.com/id/resep/24983731,Ayam panggang teflon,https://img-global.cpcdn.com/recipes/ab8f2174a...,"[1 kg ayam negri (dipotong 10), 1 buah jeruk n...","[{'text': 'Cuci bersih ayam pada air mengalir,..."
4,resep-24975281,ayam,https://cookpad.com/id/resep/24975281,Chicken Spicy Wings,https://img-global.cpcdn.com/recipes/e9ff08c91...,"[2 kg sayap ayam, Secukupnya minyak untuk meng...","[{'text': 'Cuci bersih sayap ayam, potong jadi..."


In [22]:
import re

STOP_UNITS = [
    # berat
    'g','gr','gram','kg','kilogram','mg','miligram','ons','oz','ounce','lb','lbs','pon',
    # volume
    'ml','l','lt','liter','dl','cl','cc','cup','cangkir','gelas','mangkuk','mangkok',
    'botol','kaleng','pouch','sachet','saset','bungkus','bks','pak','paket','kotak','porsi',
    # sendok & tetes
    'sdm','sendok makan','tbsp','sdt','sendok teh','tsp','sendok','sendok sayur','centong',
    'tetes','drop','drops',
    # hitung/bentuk
    'butir','buah','biji','siung','potong','lembar','helai','ruas','batang','tangkai',
    'keping','blok','ikat','ekor','kepala',
    # kira-kira
    'secukupnya','sejumput','jimpit','sejimpit','secubit','cubit','segenggam','genggam',
    'sekepal','kepal','seikat','segelas','semangkuk',
    # panjang
    'cm','mm','inci','inch',
    # other
    'jari'
]

STOP_UNITS += [
    'grm','mlm','mlt','btr','bh','pcs','bj','sdk','sm','bt','i','btir','butit'
]

# frasa noise di luar kurung
NOISE_PHRASES = [
    'potong','untuk menumis','untuk menggoreng','ukuran kecil','dipotong kecil',
    'diiris serong','diiris tipis','iris','dibalur ke daging ayam sebelum dipanggang','u/'
]

NOISE_PHRASES += [
    'skip', 'boleh diganti', 'bisa diganti', 'optional', 'opsional',
    'secukup nya', 'secukup nya', 'sckpnya', 'sckupnya',
    'kualitas bagus', 'fresh', 'baru', 'panas', 'hangat',
    'bumbu', 'bumbu halus', 'rose brand', 'rosebrand', 'sunco',
    'untuk olesan', 'untuk tumis', 'untuk mengoles', 'untuk mengolesi',
    'untuk menumis', 'buat menumis', 'utk menumis',
    'untuk menggoreng', 'utk menggoreng', 'untuk olesan', 'untuk oles',
    'untuk merebus', 'untuk blender', 'untuk campuran', 'perkedel kira kira',
    'dikocok lepas', 'kocokan', 'orak arik', 'scramble', 'ceplok', 'dadar', 'rawis', 'pelapis',
    'untuk pelapis', 'untuk isian', 'untuk tabur', 'untuk baluran',
    'yg sdh direbus', 'yang sudah direbus', 'suhu ruang', 'uk besar', 'uk kecil',
    'ukuran besar', 'ukuran sedang', 'uk sedang', 'mentah', 'matang',
    'setengah matang', 'kupas', 'dibelah', 'belah', 'potong', 'orek', 'gulung',
    'dibuat', 'tambahan', 'kasih', 'campur', 'masukkan', 'sisa', 'setetes',
    'warna kuning telur', 'pewarna kuning telur', 'pewarna makanan',
    'minyak goreng', 'uht', 'susu', 'skm', 'madu', 'mentega', 'wijen',
    'koya', 'teri', 'bawang goreng', 'kecap', 'jeruk nipis'
]

NOISE_PHRASES += [
    # konektor/filler umum
    '&amp', 'amp', 'dan', 'atau', 'serta', 'dengan', 'ke', 'yang', 'yg', 'di',
    'lalu', 'kemudian', 'jadi', 'saja', 'sih', 'nya', 'pun', 'tadi', 'agar', 'biar', 'hingga',
    'seperti', 'misal', 'contoh', 'buat', 'untuk', 'u/', 'utk', 'dengan',
    'kalau', 'kalo', 'pake', 'pakai', 'per', 'masing', 'masing jadi',
    'kurleb', 'kurang lebih',

    # deskripsi bentuk/ukuran/suhu/keadaan
    'utuh', 'besar', 'kecil', 'sedang', 'panjang', 'memanjang',
    'dingin', 'hangat', 'panas', 'setengah beku', 'setengah matang', 'matang',

    # bagian/atribut tidak relevan
    'kulitnya', 'cangkang', 'kulit', 'bagian', 'orangnya', 'warna kuning telur',

    # instruksi dapur (duplikasi dari ACTION_TERMS supaya tersapu meski di luar kurung)
    'kocok', 'kocok lepas', 'kocokan', 'kocok ampe berbuih',
    'rebus', 'direbus', 'rebus dulu', 'rebus mnt', 'rabus',
    'kukus',
    'goreng', 'digoreng',
    'tumis', 'ditumis', 'tumis sebentar',
    'sangrai',
    'orak arik', 'orek', 'ceplok', 'dadar',
    'gulung', 'belah', 'dibelah',
    'kupas', 'dikupas',
    'pecahkan',
    'pisahkan', 'pisahkan kuning dan putihnya',
    'ambil', 'ambil putihnya saja',
    'buang',
    'haluskan', 'ulek', 'tumbuk',
    'cincang',
    'sisir', 'parut', 'serut',

    # kata kerja campur/olah umum
    'campur', 'aduk rata', 'tambahkan', 'masukkan', 'mixer sampai kaku', 'sampai kaku',
    'keringkan', 'blansir', 'dikocok lepas',

    # oles/poles/egg-wash
    'bahan olesan', 'olesan', 'oles', 'polesan',
    'untuk olesan', 'untuk polesan', 'untuk bahan olesan',
    'olesan putih telur', 'olesan kuning telur',

    # label “tambahan/pelengkap/taburan”
    'bahan pendamping', 'bahan pelengkap', 'pelengkap',
    'tabur', 'untuk tabur', 'topping', 'tusuk sate',

    # pewarna/produk lain yang sering nyangkut
    'pewarna', 'pewarna kuning', 'pewarna makanan', 'plain', 'full cream',
    'kental manis', 'skm', 'uht', 'minyak', 'migor', 'minyak kuning telur',

    # frasa remeh yang sering muncul di data
    'telurnya', 'telurnya rebus', 'telurnya sudah', 'telur saya',
    'lalu hingga tebal', 'lalu dan kecil',
    'garam untuk telur', 'untuk telur',
    'bisa diganti', 'boleh diganti', 'optional', 'opsional', 'skip',

    # typo/varian aneh dari data
    'kocd,ok lepas', 'punyuh', 'putin', 'kuing',
    '=&gt', '>>', '+', '*', 'x', 'kwetiu', 'kwetiau'
]

# kata/aksi dalam kurung yang boleh dihapus (dinamis)
ACTION_TERMS = [
    'iris','iris halus','iris tipis','rajang',
    'goreng','tumis','sangrai','panggang','bakar',
    'rebus','kukus',
    'cincang','haluskan','ulek','tumbuk','geprek','memarkan',
    'serut','parut','sisir',
    'kocok','kocok lepas','suwir','campur','aduk rata','blansir','keringkan',
    'dadar','ceplok','orak arik','scramble','orek','gulung','belah','kupas'
]

ACTION_TERMS += [
    'pecahkan', 'pisahkan', 'ambil putihnya saja', 'ambil', 'buang',
    'tumis sebentar', 'rebus dulu', 'dikupas',
    'mixer sampai kaku', 'sampai kaku'
]

STOP_UNITS += [
    'menit', 'mnt', 'detik', 'dtk', 'jam', 'hari', 'hrs'
]

# (opsional) sinonim normalisasi nama bahan
SYNONYMS = {
    'cabe': 'cabai',
    'nenas': 'nanas',
    'telur rebus': 'telur',
    'telur itik': 'telur bebek',
    'telur ayam biasa': 'telur ayam',
    'telur ayam utuh': 'telur ayam',
    'telur ayam omega': 'telur ayam',
    'telur omega': 'telur ayam',
    'putin telur': 'putih telur',
    'kuing telur': 'kuning telur',
    'mi telur': 'mie telur',
    'telur mata sapi': 'telur',
    'telur pindang': 'telur',
}

def _sorted_unique_lower(xs):
    return sorted(set(x.lower() for x in xs), key=len, reverse=True)

def build_units_regex(units):
    tokens = _sorted_unique_lower(units)
    return re.compile(r'(?<!\S)(' + '|'.join(map(re.escape, tokens)) + r')(?!\S)', re.IGNORECASE)

def build_noise_phrases_regex(phrases):
    pats = [re.escape(p.lower()).replace(r'\ ', r'\s+') for p in _sorted_unique_lower(phrases)]
    return re.compile(r'(?<!\w)(?:' + '|'.join(pats) + r')(?!\w)', re.IGNORECASE)

def build_paren_action_regex(terms):
    term_pats = [r'(?:' + re.escape(t.lower()).replace(r'\ ', r'\s+') + r')' for t in _sorted_unique_lower(terms)]
    alt = '|'.join(term_pats)
    return re.compile(r'\(\s*(?:' + alt + r')(?:\s*[,;/]\s*(?:' + alt + r'))*\s*\)', re.IGNORECASE)

def build_synonym_rules(syn):
    return [(re.compile(r'(?<!\w)' + re.escape(k).replace(r'\ ', r'\s+') + r'(?!\w)', re.IGNORECASE), v)
            for k, v in syn.items()]

def rebuild_patterns():
    globals().update({
        'UNITS_RE'  : build_units_regex(STOP_UNITS),
        'NOISE_RE'  : build_noise_phrases_regex(NOISE_PHRASES),
        'PAREN_RE'  : build_paren_action_regex(ACTION_TERMS),
        'SYN_RULES' : build_synonym_rules(SYNONYMS),
        'FRACTION_RE'        : re.compile(r'\d+\s*/\s*\d+'),
        'UNICODE_FRAC_RE'    : re.compile(r'[\u00BC-\u00BE\u2150-\u215F\u2189]'),  # ¼ ½ ¾ …
        'NUMBER_RE'          : re.compile(r'[+-]?\d+(?:[.,]\d+)*'),
        'COMMA_SPLIT_RE'     : re.compile(r'\s*,\s*'),
    })

rebuild_patterns()

def _apply_synonyms(s: str) -> str:
    for rx, repl in SYN_RULES:
        s = rx.sub(repl, s)
    return s

def clean_ingredient_item(item: str) -> str:
    s = (item or '').strip().lower()
    if not s:
        return ''

    # sinonim
    s = _apply_synonyms(s)

    # hapus kurung yang HANYA berisi aksi
    s = PAREN_RE.sub(' ', s)
    # hapus kurung kosong yang tersisa
    s = re.sub(r'\(\s*\)', ' ', s)
    s = re.sub(r'\([^()]*\)', '', s)

    # angka/pecahan
    s = FRACTION_RE.sub(' ', s)
    s = UNICODE_FRAC_RE.sub(' ', s)
    s = NUMBER_RE.sub(' ', s)

    # hapus unit & frasa noise
    s = UNITS_RE.sub(' ', s)
    s = NOISE_RE.sub(' ', s)

    # bersihkan tanda baca ringan
    s = re.sub(r'[()\-–—/:;]', ' ', s)

    # rapikan spasi
    s = ' '.join(s.split())
    return s

def clean_ingredient_list(items, dedupe=True):
    cleaned = [clean_ingredient_item(x) for x in items]
    cleaned = [x for x in cleaned if x]
    if dedupe:
        cleaned = list(dict.fromkeys(cleaned))
    return cleaned

list_of_ingredients = []
# for i in range(len(df_ayam)):
#     list_of_ingredients.extend(clean_ingredient_list(df_ayam.iloc[i].ingredients))

for i in range(len(df_other)):
    list_of_ingredients.extend(clean_ingredient_list(df_other.iloc[i].ingredients))

list_of_ingredients = list(dict.fromkeys(list_of_ingredients))

In [23]:
print(list_of_ingredients[:20])
len(list_of_ingredients)

['daging sapi has dalam', 'air', 'tomat merah,', 'manis', 'bawang merah', 'aromatik', 'btg sereh, empat', 'laos lengkuas', 'lmbr daun salam', 'dihaluskan marinasi', 'bawang putih', 'kemiri', 'cabai merah keriting', 'pala bubuk', 'merica bubuk', 'gula pasir', 'garam', 'daging sapi', 'cabai giling', 'santan']


8658

In [24]:
import re
from collections import Counter

def build_search_regex(terms, allow_suffix=True, allow_hyphen=True):
    """
    terms: list kata/frasa, mis. ['minyak', 'telur', 'saus tiram']
    allow_suffix: izinkan sufiks -nya/-ku/-mu (minyaknya, telurnya)
    allow_hyphen: anggap '-' sama dengan spasi (cabai-merah ~ cabai merah)
    """
    def term_pat(t: str) -> str:
        t = t.strip()
        if not t:
            return ''
        pat = re.escape(t)
        # spasi fleksibel
        pat = pat.replace(r'\ ', r'\s+')
        # treat '-' ~ spasi (opsional)
        if allow_hyphen:
            pat = pat.replace(r'\-', r'[\-\s]+')
        return pat

    alts = [p for p in (term_pat(t) for t in terms) if p]
    if not alts:
        # regex yang gak pernah match
        return re.compile(r'^(?!)$')

    core = '(?:' + '|'.join(sorted(alts, key=len, reverse=True)) + ')'
    suffix = r'(?:-(?:nya|ku|mu)|(?:nya|ku|mu))?' if allow_suffix else ''
    # token boundary supaya tidak nyangkut di dalam kata lain
    pattern = r'(?<!\w)' + core + suffix + r'(?!\w)'
    return re.compile(pattern, re.IGNORECASE)

def filter_items_by_terms(items, terms, **rx_opts):
    rx = build_search_regex(terms, **rx_opts)
    return [s for s in items if rx.search(s)]

def index_items_by_term(items, terms, **rx_opts):
    # mapping term -> list item yang match term itu
    out = {t: [] for t in terms}
    for s in items:
        for t in terms:
            if build_search_regex([t], **rx_opts).search(s):
                out[t].append(s)
    return out

# terms = ['minyak']
# hasil = filter_items_by_terms(list_of_ingredients, terms)
# print(hasil)

# kalau mau per-term:
idx = index_items_by_term(list_of_ingredients, ['telur'])
for k, v in idx.items():
    print(k, '->', v)

# # hitung frekuensi hasil
# counts = Counter(filter_items_by_terms(list_of_ingredients, ['telur']))
# print(counts)


telur -> ['telur', 'telur asin', 'telur ayam', 'kuning telur', 'putih telur', 'telur, telur', 'telur,', 'telur, menjadi', 'telur puyuh', 'kuning telur dari bahan adonan daging', 'telur ayam ,', 'telur puyuh telur ayam', 'tahu sutra tahu telur', 'telur puyuh, , , sisihkan', 'telur balado', 'mie telur', 'telur ayam, ,', 'telur .lepas', 'garam telur', 'telur ayam,', 'telur, , , , sisihkan', 'air es merendam telur', 'telur bebek', 'telur ayam kampung', 'telur kuning', 'telur putih', 'telur ,', 'telur antero', 'telur, resep aslinya telur', 'mixet bahan kuning telur', 'bahan adonan kuning telur', 'putih telur perekat lumpia.', 'telur , kuning telur', 'kuning telur putih telur', 'sesuai selera telur,tahu putih,kol,kentang,pare', 'kol, kentang telur', 'tepung terigu serbaguna, maizena, telur, garam, air cair, sayur', 'telur , jika', 'kuning telur, putihnya', 'telur..', 'telur ayam, , telur dipotong', 'telur * kehabisan', 'kuning telur asin', 'butuh telur didadar', 'air telur puyuh', 'telur, du